In [ ]:
import tensorflow as tf
import tensorflow.keras.losses as losses
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
import tensorflow.keras.optimizers as optimizers
!pip install -q -U keras-tuner
from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters
import time
LOG_DIR = f"{int(time.time())}"
from keras.layers import BatchNormalization
from tempfile import TemporaryFile
import numpy as np
import matplotlib.pyplot as plt
import os
import tensorflow.keras.regularizers as regularizers
import cv2
import random
from PIL import Image
import matplotlib.patches as patches
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from google.colab import drive
import time
drive.mount('/content/drive/')
dataset_dir = "/content/drive/My Drive/dataset_aircraft"
 
training_data = []
test_data = []
RESIZE_VALUE = 130
root_path = "/content/drive/My Drive/dataset_aircraft/"

In [ ]:
drive.mount('/content/drive/')
dataset_dir = "/content/drive/My Drive/dataset_aircraft"

In [61]:
def image_process():
  #image processing (dataset image randomizer)
  plane = "c/"
  datagen = ImageDataGenerator(
          rotation_range=360,
          width_shift_range=0,
          height_shift_range=0,
          shear_range=0,
          zoom_range=0.0,
          horizontal_flip=True,
          fill_mode='nearest')
  
  selected_path = root_path+plane
  for img_ in os.listdir(selected_path):
    img = load_img(selected_path+img_)  # this is a PIL image
    x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
    x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)

    # the .flow() command below generates batches of randomly transformed images
    # and saves the results to the `preview/` directory
    i = 0
    for batch in datagen.flow(x, batch_size=1,
                              save_to_dir=root_path+plane, save_prefix=img_, save_format='jpeg'):
        i += 1
        if i > 20: # the number of image generation (amendments) per selected picture
            break
image_process()

In [ ]:
# outputs the names of the files currently in the google drive file (dataset) 
types_path = "/content/drive/My Drive/dataset_aircraft/TYPE-NAMES.txt"
aircraft_types = []
f = open(types_path, "r")
for types in f.read().split():
  aircraft_types.append(types.replace('"',''))
print(len(aircraft_types))
f.close()
print("Selected aircraft files:",aircraft_types)
print("Files in dataset (Google drive):",os.listdir(root_path))

In [ ]:
def create_training_data():
    # function iterates through each file in the google drive dataset and converts each image into an array
  for type in aircraft_types:
    selected_path = root_path + type
    class_num = aircraft_types.index(type)
    for img in os.listdir(selected_path):
      try:
        image = Image.open(os.path.join(selected_path,img))
        img_array = np.array(image)
        aircraft_img = cv2.resize(img_array, (RESIZE_VALUE, RESIZE_VALUE))
        training_data.append([aircraft_img, class_num])
      except Exception as e:
        print("error:",selected_path,image)
 
create_training_data()
random.shuffle(training_data)
print(len(training_data))
print(training_data[0][1])
plt.imshow(training_data[0][0])

In [ ]:
# saves all generated arrays of images to a file
np.savez_compressed(root_path+'primary_training_dataset.npz', a=training_data)

In [153]:
def load_training_data():
  #LOAD TRAINING DATA INTO ARRAY FROM FILE
  loaded = np.load(root_path+"primary_training_dataset.npz", allow_pickle=True)
  training_data = loaded['a']
  for x in range(len(training_data)):
    if (training_data[x][0].shape[2] == 4):
      training_data[x][0] = cv2.cvtColor(training_data[x][0], cv2.COLOR_BGRA2BGR)
  return training_data
training_data = load_training_data()
random.shuffle(training_data)
print(len(training_data))

6272


In [ ]:
def load_testing():
  #LOAD TESTING DATA INTO ARRAY FROM FILE
  loaded = np.load(root_path+"primary_training_dataset.npz", allow_pickle=True)
  training_data = loaded['a']
  for x in range(len(training_data)):
    if (training_data[x][0].shape[2] == 4):
      training_data[x][0] = cv2.cvtColor(training_data[x][0], cv2.COLOR_BGRA2BGR)
  return training_data
testing_data = load_testing()
random.shuffle(testing_data)
print(len(testing_data))

In [ ]:
x_train = []
y_train = []
 
#x_test = []
#y_test = []
 
#for data, label in testing_data:
#  x_test.append(data)
#  y_test.append(label)
 
for data, label in training_data:
  x_train.append(data)
  y_train.append(label)
 
x_train = np.asarray(x_train)
x_train = np.array(x_train).reshape(-1, RESIZE_VALUE, RESIZE_VALUE, 3)
y_train = np.asarray(y_train)
 
#x_test = np.asarray(x_test)
#x_test = np.array(x_test).reshape(-1, RESIZE_VALUE, RESIZE_VALUE, 3)
#y_test = np.asarray(y_test)

x_train = x_train / 255
#x_test = x_test / 255
print(len(x_train))
#print(len(x_test))

In [210]:

model = keras.models.load_model(root_path+"Model V6")

In [ ]:
model.summary()

In [ ]:
def build_model_MAC():
    # main model
    model = Sequential()
    model.add(Conv2D(32, (3, 3), input_shape = (RESIZE_VALUE,RESIZE_VALUE,3)))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))


    
    model.add(Flatten())
    
    model.add(Dense(450))
    model.add(Activation('relu'))
    model.add(Dropout(.65))
    
    model.add(Dense(len(aircraft_types)))
    model.add(Activation("softmax"))

    opt = keras.optimizers.Adam(learning_rate=0.0001)
    model.compile(loss='sparse_categorical_crossentropy',
                    optimizer=opt,
                    metrics=['acc'])
  
    return model

model = build_model_MAC()
epochs = 30
x = np.arange(epochs) + 1
history_ = model.fit(x_train, y_train, epochs=epochs, validation_split=.30, batch_size=64)
plt.plot(x,history_.history['acc'])
plt.plot(x,history_.history['val_acc'])
plt.xlabel("Epochs");
plt.ylabel("Accuracy")
plt.legend(['Training Accuacy', 'Cross Validation Accuracy']);

fig, ax = plt.subplots()
ax.plot(x,history_.history['loss'])
ax.plot(x,history_.history['val_loss'])
plt.xlabel("Epochs");
plt.ylabel("Loss")
plt.legend(['Training Loss', 'Cross Validation Loss']);


In [ ]:
model.save(root_path+"Model V6")

In [ ]:
def testSingleAircraft():
    # function takes a single image of an aircraft and classifies the aircraft category
    img = "C-17.JPG"
    image = Image.open(os.path.join(root_path+"/Testing Images/",img))
    img_array = np.array(image)
    aircraft_img = cv2.resize(img_array, (RESIZE_VALUE, RESIZE_VALUE))
    patch = np.expand_dims(aircraft_img, axis=0)
    plt.imshow(aircraft_img)
    x = model.predict_classes(patch)
    print ("True value:", img)
    print("Identified as:",aircraft_types[x[0]],"\n")

In [ ]:
def testMultipleAircraft():
    #function takes a wide picture consisting of multiple aircrafts and marks aircrafts
    img = "1.JPG"
    image = Image.open(os.path.join(root_path+"/Testing Images/",img))
    img_array = np.array(image)
    img_shape = img_array.shape
    aircraft_img = cv2.resize(img_array, (img_shape[1], img_shape[0]))
    test_img =  cv2.cvtColor(aircraft_img, cv2.COLOR_BGRA2BGR)
    plt.imshow(test_img)
    ax = plt.gca()
    pred_dist = np.arange(len(aircraft_types))
    classif = []
    detectCount = 0;
    counter = 0
    # set length and width of patch
    length = 80
    width = 80
    # do not show list
    dshow = ["BareLand"]
    # iterate through the entire image and predict each patch
    for i in range(0,test_img.shape[0], 30):
      for j in range (0,test_img.shape[1], 30):
        if (j+length < test_img.shape[1] and i+width < test_img.shape[0]):
          raw_patch = test_img[i:i+length, j:j+width]
          aircraft_img = cv2.resize(raw_patch, (RESIZE_VALUE, RESIZE_VALUE))
          patch = np.expand_dims(aircraft_img, axis=0)
          ident = model.predict_classes(patch)
          pred_dist[ident[0]] += 1
          u = aircraft_types[ident[0]]
          # do not mark patch if patch is classified as bareland (background)
          if (u not in dshow):
            # mark the identified patch with a red rectangle ROI, region of interest
            ax.add_patch(patches.Rectangle((j, i), (j+length)-j, (i+width)-i, facecolor='none',edgecolor='red', linewidth=.5))

    plt.show()

testMultipleAircraft()

In [ ]:
def build_model(hp):
    # this function uses hyperparameter method provided by keras to fine tune a model 
  model = Sequential()
 
  model.add(Conv2D(hp.Int('conv1', 32,64,12), (3, 3), input_shape = (RESIZE_VALUE,RESIZE_VALUE,3)))
  model.add(Activation("relu"))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  
  model.add(Conv2D(hp.Int('conv2', 32,64,12), (3, 3)))
  model.add(Activation("relu"))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  
  model.add(Conv2D(hp.Int('conv3', 32,64,12), (3, 3)))
  model.add(Activation("relu"))
  model.add(MaxPooling2D(pool_size=(2, 2)))



 
  model.add(Flatten())
  
  model.add(Dense(hp.Int('dense1', 400,1000,100)))
  model.add(Activation('relu'))
  model.add(
            Dropout(rate=hp.Float(
                'dropout_1',
                min_value=0.40,
                max_value=0.7,
                default=0.50,
                step=0.10,
            )))
 
  model.add(Dense(len(aircraft_types)))
  model.add(Activation("softmax"))

  opt = keras.optimizers.Adam(learning_rate=0.0001)
  model.compile(loss='sparse_categorical_crossentropy',
                    optimizer=opt,
                    metrics=['acc'])
  
  return model

def tuner():
    tuner = RandomSearch(
    build_model,
    objective="val_acc",
    max_trials=20,
    executions_per_trial = 1,
    directory = LOG_DIR
    )

    tuner.search(x=x_train,
             y=y_train,
             epochs=50,
             batch_size=80,
             validation_split=.35)

#tuner()